# 7.01


# Loading libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)


# Reading data

In [ ]:
data = pd.read_csv('learningSet.csv')

In [ ]:
print(data.shape)
data.head()

Column "TARGET_B" encodes if the customer answered or not the call

In [ ]:
data['TARGET_B'].value_counts()

Column "TARGET_D" provide the donation amount.

In [ ]:
data['TARGET_D'].describe()

In [ ]:
data[data['TARGET_B']>0]['TARGET_D'].describe()

To solve this problem we will first build a classification model to predict who will more likely respond and then for those respondents, we will build a regression model to predict the donation amount.

Then we can use the cost matrix to calculate the total benefit from the donations (remember that the company has to do a mailing champaing and each envelope costs some money).

Some of the challenges with the dataset are as follows:

* Large number of features: The data set has over 450 features. Hence selecting the right features for the model is very critical and at the same time it is not easy as the same traditional ways of removing features is not effective given the large number of features. Apart from feature selection, feature extraction (creating your own features using the existing features) is also not easy in this case.

* Sparsity of the dataset: There are a lot of features with a large number of null values.

* Data imbalance: For developing a classification, there is a huge imbalance in the training dataset with only approximately 5000 values for one category as compared to cover 95,000 instances for the other category.

# Review data cleaning process

There are a lot of columns that have a very high percentage of null values. It is a highly sparse dataset.
We can decide on a threshold and then remove those variables. There is no rule of thumb to decide on this threshold value.
Sometimes it can as low as 25%-30%. And sometimes in some data sets you might find that even though there are more than 50% missing values in a column, you might have to include that variable in your analysis.
A lot of it depends on the business context as well. In this case we will take this threshold to be 25% and then check the definitions of the columns filtered, to see if there is any column that we might want to keep.

In [ ]:
data.shape[0]

In [ ]:
len(data)

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

Which columns have NA's?

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

From the list above that includes the columns that have over 25% null values, you discussed with your manager you were told that the following columns are important -> **wealth1, wealth2**
We will remove these variables from the above list
RDATE3, RAMNT_3 are important but they have too many null values

# Activity

Create a function that takes a dataframe as an input and a percentual threshold (default value = 0.25) and returns a list of columns with null values greater than the specified threshold

In [ ]:
# Define the function here
def null_percent_col_list (df, threshold:float = 0.25)-> list:
    df2 = df.copy()
    nulls_percent_df = pd.DataFrame(df2.isna().sum()/len(df2)).reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>threshold]['column_name'].tolist()
    #columns_above_threshold =[columns_above_threshold]
    return columns_above_threshold


In [ ]:
# Use your function here
cols_to_drop = null_percent_col_list(data)

In [ ]:
print(cols_to_drop)

In [ ]:
len(cols_to_drop)

In [ ]:
# End of the activity

# More data cleaning

In [ ]:
data.head()

We can see that there are a lot of columns that have blank spaces which represent no value in this case.
They were not identified as null values by python as they are empty spaces that are read as character values by
python. We will replace those values by NaNs and repeat the analysis

Before we do that we will replace the blank values from the column "MAILCODE" by "A" which would mean the address is okay (pl check the definition of the variable in the description)

In [ ]:
data['MAILCODE'].value_counts(dropna=False)

In [ ]:
data['MAILCODE'].unique()

In [ ]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))
#data['MAILCODE'] = np.where(data['MAILCODE'] == " ", "A",data['MAILCODE'] )
#data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x if (x != " ") else "A")
# function(arg1)
#df[col] = list(map(function,df[col]))
# function(arg1, arg2, arg3)
#df[col] = list(map(function,df[col],[arg2]*len(df), [arg3]*len(df)))

In [ ]:
# Now we can replace the rest space characters with np.NaN

In [ ]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

In [ ]:
data.head()

# Activity

Use the previous function to obtain the columns with null values greater than 0.25 and store those columns in a list called "drop_columns_list"

In [ ]:
# Your code here
drop_columns_list = null_percent_col_list(data)

In [ ]:
drop_columns_list

In [ ]:
len(drop_columns_list)

In [ ]:
# End of activity

We would again repeat the same exercise as the last time. We will discuss it with the team, manager, and/or other stakeholders to see which columns we need to retain here.

Like the last time we will keep the following columns. **wealth1, wealth2 along with these VETERANS, SOLIH**

Remove those column's names from the list of columns to drop.

In [ ]:
def remove_columns(df, thr = 0.25, blacklist=None):
    ...
    if ( blacklist != None):
        drop_columns_list = [ col for col in drop_columns_list if col not in blacklist ]

In [ ]:
drop_columns_list = [ col for col in drop_columns_list if col not in ['WEALTH1','WEALTH2','VETERANS','SOLIH'] ]

In [ ]:
len(drop_columns_list)

# X-y split

Since we have a huge number of features, it would be easier to work independently on numerical features and categorical features.

For the target variables, for now we will retain them both together. But later, we will build a classification model first where we would need the column TARGET_B only.

In [ ]:
data.TARGET_B.value_counts()

In [ ]:
Y = data[['TARGET_B', 'TARGET_D']]
Y.head()

In [ ]:
numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])
numerical.head()

In [ ]:
numerical.shape

In [ ]:
categorical = data.select_dtypes([object])
categorical.head()

In [ ]:
categorical.shape

# Working with categorical columns

We will work with the categorical features first. Look at the columns one by one. Some of the operations which we will perform are:

- Replace null values with the most occurring categories
- Reduce the number of categories in a column by grouping

It is important to note that some columns are defined by python as categorical/object types. There might be other columns defined as numerical that we want as categorical. We will look them later when we were working on numerical types.

In [ ]:
categorical.isna().sum()

# Categorical Variables

Here we will try to reduce the number of categories. An ideal way would have been to group the states into
regions. But in this case we will group all the states with counts less than 2500 into one category "other"

In [ ]:
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()


df.columns = ['state', 'count']
other_states = list(df[df['count']<2500]['state'])

# other_states = ['CA','AR','MD']
def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x

# clean_states(x,other_states) for x, other_states in zip(categorical['STATE'].values,other_states)
categorical['STATE'] = list(map(clean_state, categorical['STATE']))
#categorical['STATE'] = categorical['STATE'].apply(clean_state)

In [ ]:
display(categorical['STATE'].nunique())
categorical['STATE'].value_counts()

# 7.01 Lab | Revisiting Machine Learning Case Study

Lab instructions can be found [here](https://github.com/isg75/lab-revisiting-machine-learning/tree/master)

# Lab | Revisiting Machine Learning Case Study

- In this lab, you will use `learningSet.csv` file which we used during the class. 

### Instructions

Complete the following steps on the categorical columns in the dataset:

- Check for null values in all the columns
- Create a new empty list called `drop_list`. We will append to this list a set of columns to be dropped later. Add the following columns to this:
    - `OSOURCE` - symbol definitions not provided, too many categories
    - `ZIP` - we are including states already
- Identify columns that have over 85% missing values and add them to the previous list.
- Remove the columns included in the `drop_list` from the dataframe
- Now, reduce the number of categories in the column `GENDER`. The column should only have either "M" for males, "F" for females, and "other" for all the rest
    - Note that there are a few null values in the column. We will first replace those null values using the code below:

    ```python
    print(categorical['GENDER'].value_counts())
    categorical['GENDER'] = categorical['GENDER'].fillna('F')

In [ ]:
# Check for null values in all the columns
categorical.isnull().sum()


In [ ]:
# Create a new empty list called `drop_list`. We will append to this list a set of columns to be dropped later. Add the following columns to this:
#    - `OSOURCE` - symbol definitions not provided, too many categories
#    - `ZIP` - we are including states already
drop_list = []
[drop_list.append(x)for x in ['OSOURCE', 'ZIP']]
#drop_list.append('ZIP')
drop_list

In [ ]:
#  Identify columns that have over 85% missing values and add them to the previous list.

columns_above_85 = null_percent_col_list(categorical, 0.85)
display(len(columns_above_85))

#'OSOURCE' in columns_above_threshold
#'ZIP' in columns_above_threshold
drop_list = [x for x in drop_list if x not in columns_above_85] + columns_above_85
drop_list
len(drop_list)


In [ ]:
drop_list

In [ ]:
#  Remove the columns included in the `drop_list` from the dataframe
categorical = categorical.drop(columns = drop_list) 


In [ ]:
categorical

In [ ]:
# - Now, reduce the number of categories in the column `GENDER`. The column should only have either "M" for males, "F" for females, and "other" for all the rest
#    - Note that there are a few null values in the column. We will first replace those null values using the code below:
#    ```python
#    print(categorical['GENDER'].value_counts())
#    categorical['GENDER'] = categorical['GENDER'].fillna('F')

display(categorical.GENDER.isnull().sum())
display(categorical.GENDER.value_counts(dropna=False))
display(categorical.GENDER.unique())
categorical['GENDER'] = categorical['GENDER'].fillna('F')
categorical["GENDER"] = categorical["GENDER"].apply(lambda x: x if x in ['F', "M"] else 'other')
display(categorical.GENDER.value_counts())

In [ ]:
categorical.columns